In [97]:
import os

In [109]:
class LZ78_4byte:
    
    def compress(self, origin_file, compressed_file):
        import struct 
        f = open(origin_file, 'r')
        data = f.readlines()
        f.close()

        data = ''.join(data)
        encoded_data = self.encoding(data)
        
        maxidx = max([t[0] for t in encoded_data])
        
        if maxidx < 255:
            bintype = 'Bc'
        elif maxidx < 65535:
            bintype = 'Hc'
        elif maxidx < 16777215:
            bintype = '4byte'
        else:
            bintype = 'Ic'
        
        binfile = open(compressed_file, 'wb')
        binfile.write(struct.pack('c', bintype[:1].encode()))
        for idx, ch in encoded_data:
            if len(ch) == 0:
                break
            if bintype == '4byte':
                data = struct.pack('Ic', idx, ch.encode())
                binfile.write(b''.join([data[0:3], data[4:]])) 
            else:
                data = struct.pack(bintype, idx, ch.encode())
                binfile.write(data) 
        binfile.close()
        
        return encoded_data
        
    def decompress(self, compressed_file, decompressed_file):
        import struct 
        binfile = open(compressed_file, 'rb')
        a = binfile.read(1)
        bintype = struct.unpack('c', a)[0].decode()
        encoded_data = []

        while True:
            if bintype == '4':
                binary = binfile.read(struct.calcsize('Ic') - 1)
                if binary == b'': break 
                encoded_data.append(struct.unpack('Ic', b''.join([binary[0:3], b'\x00', binary[3:]])))
            else:
                binary = binfile.read(struct.calcsize('%sc' % bintype))
                if binary == b'': break 
                encoded_data.append(struct.unpack('%sc' % bintype, binary))
            
        encoded_data = [(d[0], d[1].decode()) for d in encoded_data]
        
        decoded_data = self.decoding(encoded_data)
        
        f = open(decompressed_file, 'w')
        f.write(decoded_data)
        f.close()
        
    def encoding(self, data):
        import collections
        encode_dict = collections.OrderedDict()
        out = []
        out2 = []
        key = ''
        
        for i, c in enumerate(data):
            key += c
            if key not in encode_dict:
                out.append((encode_dict[key[:-1]] if len(key) > 1 else 0, c))
                encode_dict[key] = len(encode_dict)+1
                key = ''
                
        if key != '': out.append((encode_dict[key], ''))

        return out, encode_dict
    
    def decoding(self, data):
        d = []
        p = ''

        for (w, c) in data: d.append(c if w == 0 else d[w-1] + c)

        return ''.join(d)

In [102]:
lz78 = LZ78_4byte()
ed = lz78.compress('infile.txt', 'compress.lz')
lz78.decompress('compress.lz', 'restore.txt')

In [100]:
print('File name: infile.txt, \nOrigin file size: %sByte, \nCompressed size: %sByte, \nCompression ratio: %f%%' % 
('{:,}'.format(os.path.getsize('./infile.txt')), '{:,}'.format(os.path.getsize('./compress.lz')), 
 os.path.getsize('./compress.lz')/os.path.getsize('./infile.txt')*100))

File name: infile.txt, 
Origin file size: 1,555,051Byte, 
Compressed size: 1,016,949Byte, 
Compression ratio: 65.396505%


In [112]:
data = ['ABBCBCABABCAABCAAB', 'BABAABRRRA', 'AAAAAAAAA']

for origin_text in data[:1]:
    lz78_ = LZ78_4byte()
    encoded_text, edict = lz78_.encoding(origin_text)
    decoded_text = lz78_.decoding(encoded_text)
    print(origin_text, encoded_text, decoded_text, origin_text == decoded_text)
    print('--------')

ABBCBCABABCAABCAAB [(0, 'A'), (0, 'B'), (2, 'C'), (3, 'A'), (2, 'A'), (4, 'A'), (6, 'B')] ABBCBCABABCAABCAAB True
--------


In [113]:
edict

OrderedDict([('A', 1),
             ('B', 2),
             ('BC', 3),
             ('BCA', 4),
             ('BA', 5),
             ('BCAA', 6),
             ('BCAAB', 7)])

In [114]:
encoded_text

[(0, 'A'), (0, 'B'), (2, 'C'), (3, 'A'), (2, 'A'), (4, 'A'), (6, 'B')]

In [131]:
a = struct.pack('Ic', 100000, 'c'.encode())

In [147]:
for b in a:
    print(b)
print(a)

160
134
1
0
99
b'\xa0\x86\x01\x00c'


In [145]:
(256 ** 2) + (256 * 134) + 160

100000